# 🐝 Bee Positive: UK Biodiversity Indicators — Machine Learning Analysis
---
## 1. Introduction & Project Overview

**Client**: Bee Positive (Non-Government Organisation)  
**Objective**: Develop a proof-of-concept ML application to analyse UK Biodiversity Indicator datasets, with the goal of planning, forecasting, and evaluating actions to support pollinating insect populations and biodiversity.

### Datasets Analysed:
1. **Pollinating Insects** — Occupancy indices for bees & hoverflies (1980–2024)
2. **Habitat Connectivity** — Functional connectivity indices for butterflies & birds (1985–2012)
3. **Agri-Environment Schemes** — Land area under environmental schemes by country (1992–2022)
4. **Plants of the Wider Countryside** — Plant abundance indices by habitat type (2015–2024)
5. **Insects of the Wider Countryside** — Butterfly abundance trends (1976–2024)

### Methodology:
- **AI Search/Optimisation**: Genetic Algorithm for optimal data imputation strategy selection
- **Supervised ML**: Regression models to predict pollinator population trends
- **Unsupervised ML**: K-Means clustering to identify species groupings by trend patterns


## 2. Environment Setup & Library Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import xml.etree.ElementTree as ET
import warnings
import os
import random

from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import (mean_absolute_error, mean_squared_error, r2_score,
                             silhouette_score, silhouette_samples)
from sklearn.impute import KNNImputer

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

print("All libraries imported successfully.")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")


## 3. Data Loading & Initial Exploration

We load all five datasets from their Excel files. The Plants dataset requires a custom XML parser due to a file format incompatibility with the openpyxl library — the workbook uses a non-standard OOXML namespace declaration, which is a data quality challenge we must address.


In [ ]:
# Define file paths
BASE_PATH = '.'
FILES = {
    'pollinating': 'UK-BDI-2025-pollinating-insects.xlsx',
    'habitat': 'UK-BDI-2025-habitat-connectivity.xlsx',
    'agri': 'UK-BDI-2025-agri-environment-schemes.xlsx',
    'plants': 'UK-BDI-2025-plants-wider-countryside.xlsx',
    'insects': 'UK-BDI-2025-insects-wider-countryside.xlsx'
}

# Verify all files exist
for key, fname in FILES.items():
    path = os.path.join(BASE_PATH, fname)
    exists = os.path.exists(path)
    size = os.path.getsize(path) if exists else 0
    print(f"{'✅' if exists else '❌'} {key}: {fname} ({size:,} bytes)")


### 3.1 Pollinating Insects Dataset

In [ ]:
# Sheet 1: Yearly occupancy index for ALL pollinators (1980-2024)
poll_yearly = pd.read_excel(os.path.join(BASE_PATH, FILES['pollinating']), sheet_name='1', header=None)
poll_yearly = poll_yearly.iloc[3:].reset_index(drop=True)
poll_yearly.columns = ['Year', 'Indicator_Mean_Occupancy', 'CI_Lower', 'CI_Upper']
poll_yearly = poll_yearly.apply(pd.to_numeric, errors='coerce')
print("Pollinating Insects - Yearly Occupancy Index (All Pollinators):")
print(f"Shape: {poll_yearly.shape}")
display(poll_yearly.head(10))
print(f"\nYear range: {poll_yearly['Year'].min():.0f} to {poll_yearly['Year'].max():.0f}")
print(f"Missing values:\n{poll_yearly.isnull().sum()}")


In [ ]:
# Sheet 2: Long/short term trend summary (all pollinators)
poll_trends = pd.read_excel(os.path.join(BASE_PATH, FILES['pollinating']), sheet_name='2', header=None)
poll_trends = poll_trends.iloc[3:].reset_index(drop=True)
poll_trends.columns = ['Trend', 'Period', 'Num_Species', 'Pct_Species']
poll_trends[['Num_Species', 'Pct_Species']] = poll_trends[['Num_Species', 'Pct_Species']].apply(pd.to_numeric, errors='coerce')
print("Pollinator Trend Summary (All Pollinators):")
display(poll_trends)


In [ ]:
# Sheet 3: Yearly occupancy for WILD BEES
poll_bees = pd.read_excel(os.path.join(BASE_PATH, FILES['pollinating']), sheet_name='3', header=None)
poll_bees = poll_bees.iloc[3:].reset_index(drop=True)
poll_bees.columns = ['Year', 'Bee_Occupancy', 'Bee_CI_Lower', 'Bee_CI_Upper']
poll_bees = poll_bees.apply(pd.to_numeric, errors='coerce')

# Sheet 5: Yearly occupancy for HOVERFLIES
poll_hover = pd.read_excel(os.path.join(BASE_PATH, FILES['pollinating']), sheet_name='5', header=None)
poll_hover = poll_hover.iloc[3:].reset_index(drop=True)
poll_hover.columns = ['Year', 'Hoverfly_Occupancy', 'Hoverfly_CI_Lower', 'Hoverfly_CI_Upper']
poll_hover = poll_hover.apply(pd.to_numeric, errors='coerce')

# Sheet 7: Species list
poll_species = pd.read_excel(os.path.join(BASE_PATH, FILES['pollinating']), sheet_name='7', header=None)
poll_species = poll_species.iloc[3:].reset_index(drop=True)
poll_species.columns = ['Taxon_Group', 'Species']

print(f"Wild Bees yearly data: {poll_bees.shape} | Year range: {poll_bees['Year'].min():.0f}-{poll_bees['Year'].max():.0f}")
print(f"Hoverflies yearly data: {poll_hover.shape} | Year range: {poll_hover['Year'].min():.0f}-{poll_hover['Year'].max():.0f}")
print(f"\nSpecies list: {len(poll_species)} species")
print(f"Taxon groups: {poll_species['Taxon_Group'].value_counts().to_dict()}")


### 3.2 Habitat Connectivity Dataset

In [ ]:
# Sheet 1: Composite butterfly connectivity index (1985-2012)
hab_butterfly = pd.read_excel(os.path.join(BASE_PATH, FILES['habitat']), sheet_name='1', header=None)
hab_butterfly = hab_butterfly.iloc[3:].reset_index(drop=True)
hab_butterfly.columns = ['Year', 'Unsmoothed_Index', 'Smoothed_Index', 'Smoothed_CI_Lower', 'Smoothed_CI_Upper']
hab_butterfly = hab_butterfly.apply(pd.to_numeric, errors='coerce')

# Sheet 4: Composite bird connectivity index (1985-2012)
hab_bird = pd.read_excel(os.path.join(BASE_PATH, FILES['habitat']), sheet_name='4', header=None)
hab_bird = hab_bird.iloc[4:].reset_index(drop=True)
hab_bird.columns = ['Year', 'Bird_Unsmoothed', 'Bird_Smoothed', 'Bird_CI_Lower', 'Bird_CI_Upper']
hab_bird = hab_bird.apply(pd.to_numeric, errors='coerce')

# Sheet 3: Individual butterfly species connectivity trends
hab_species = pd.read_excel(os.path.join(BASE_PATH, FILES['habitat']), sheet_name='3', header=None)
hab_species = hab_species.iloc[4:].reset_index(drop=True)
hab_species.columns = ['Common_Name', 'Species_Name', 'Long_Term_Trend', 'Early_Short_Term', 'Late_Short_Term']

# Sheet 7 & 8: Mean habitat connectivity values
hab_woodland = pd.read_excel(os.path.join(BASE_PATH, FILES['habitat']), sheet_name='7', header=None)
hab_woodland = hab_woodland.iloc[3:].reset_index(drop=True)
hab_woodland.columns = ['Year', 'Habitat_Type', 'Mean_Connectivity']

hab_scotland = pd.read_excel(os.path.join(BASE_PATH, FILES['habitat']), sheet_name='8', header=None)
hab_scotland = hab_scotland.iloc[3:].reset_index(drop=True)
hab_scotland.columns = ['Year', 'Habitat_Type', 'Mean_Connectivity']

print("Habitat Connectivity - Butterfly Composite Index:")
print(f"Shape: {hab_butterfly.shape} | Year range: {hab_butterfly['Year'].min():.0f}-{hab_butterfly['Year'].max():.0f}")
display(hab_butterfly.head())
print(f"\nBird Composite: {hab_bird.shape}")
print(f"Individual butterfly species: {len(hab_species)} species")
print(f"Missing values (butterfly): {hab_butterfly.isnull().sum().to_dict()}")


### 3.3 Agri-Environment Schemes Dataset

In [ ]:
# Sheet 1: Higher-level schemes
agri_higher = pd.read_excel(os.path.join(BASE_PATH, FILES['agri']), sheet_name='1', header=None)
agri_higher = agri_higher.iloc[5:].reset_index(drop=True)
agri_higher.columns = ['Year', 'Country', 'Area_MHa']
agri_higher['Year'] = pd.to_numeric(agri_higher['Year'], errors='coerce')
agri_higher['Area_MHa'] = pd.to_numeric(agri_higher['Area_MHa'], errors='coerce')
agri_higher = agri_higher.dropna(subset=['Year'])

# Sheet 2: Entry-level schemes
agri_entry = pd.read_excel(os.path.join(BASE_PATH, FILES['agri']), sheet_name='2', header=None)
agri_entry = agri_entry.iloc[5:].reset_index(drop=True)
agri_entry.columns = ['Year', 'Country', 'Area_MHa']
agri_entry['Year'] = pd.to_numeric(agri_entry['Year'], errors='coerce')
agri_entry['Area_MHa'] = pd.to_numeric(agri_entry['Area_MHa'], errors='coerce')
agri_entry = agri_entry.dropna(subset=['Year'])

print("Agri-Environment Schemes - Higher Level:")
print(f"Shape: {agri_higher.shape} | Years: {agri_higher['Year'].min():.0f}-{agri_higher['Year'].max():.0f}")
print(f"Countries: {agri_higher['Country'].unique().tolist()}")
display(agri_higher.head(8))

print(f"\nEntry Level: {agri_entry.shape} | Years: {agri_entry['Year'].min():.0f}-{agri_entry['Year'].max():.0f}")


### 3.4 Plants of the Wider Countryside Dataset

> **Note**: This dataset uses a non-standard OOXML namespace in its workbook.xml, rendering it unreadable by openpyxl. We implement a custom XML parser to extract the data directly from the xlsx zip archive. This is documented as a data quality challenge.


In [ ]:
def parse_plants_xlsx(filepath):
    """Custom parser for the Plants dataset which uses non-standard OOXML namespace."""
    with zipfile.ZipFile(filepath) as z:
        # Extract shared strings (text values)
        shared_strings = []
        if 'xl/sharedStrings.xml' in z.namelist():
            ss_raw = z.read('xl/sharedStrings.xml').decode('utf-8')
            ss_root = ET.fromstring(ss_raw)
            for elem in ss_root.iter():
                if elem.tag.endswith('}t') or elem.tag == 't':
                    shared_strings.append(elem.text if elem.text else '')
        
        results = {}
        for sheet_idx, sheet_name in [(4, 'abundance'), (5, 'species_list')]:
            sf = f'xl/worksheets/sheet{sheet_idx}.xml'
            if sf not in z.namelist():
                continue
            content = z.read(sf).decode('utf-8')
            sheet_root = ET.fromstring(content)
            
            rows = []
            for row_elem in sheet_root.iter():
                if row_elem.tag.endswith('}row'):
                    row_data = {}
                    for cell in row_elem:
                        if cell.tag.endswith('}c'):
                            ref = cell.get('r', '')
                            col_letter = ''.join(c for c in ref if c.isalpha())
                            cell_type = cell.get('t', '')
                            val_elem = None
                            for child in cell:
                                if child.tag.endswith('}v'):
                                    val_elem = child
                                    break
                            if val_elem is not None:
                                if cell_type == 's':
                                    idx = int(val_elem.text)
                                    value = shared_strings[idx] if idx < len(shared_strings) else None
                                else:
                                    try:
                                        value = float(val_elem.text)
                                    except ValueError:
                                        value = val_elem.text
                            else:
                                value = None
                            row_data[col_letter] = value
                    if row_data:
                        rows.append(row_data)
            results[sheet_name] = rows
    return results

plants_raw = parse_plants_xlsx(os.path.join(BASE_PATH, FILES['plants']))

# Parse abundance data (skip header rows 0-5, data starts at row 5 with headers at row 5)
abundance_rows = plants_raw['abundance'][5:]  # Skip metadata rows
plants_abundance = pd.DataFrame(abundance_rows)
plants_abundance = plants_abundance.rename(columns={'A': 'Habitat', 'B': 'Year', 'C': 'Unsmoothed_Index',
                                                      'D': 'Lower_95_CI', 'E': 'Upper_95_CI'})
plants_abundance = plants_abundance[['Habitat', 'Year', 'Unsmoothed_Index', 'Lower_95_CI', 'Upper_95_CI']]
for col in ['Year', 'Unsmoothed_Index', 'Lower_95_CI', 'Upper_95_CI']:
    plants_abundance[col] = pd.to_numeric(plants_abundance[col], errors='coerce')

# Parse species list
species_rows = plants_raw['species_list'][4:]
plants_species = pd.DataFrame(species_rows)
plants_species = plants_species.rename(columns={'A': 'Habitat', 'B': 'Species', 'C': 'Common_Name'})
plants_species = plants_species[['Habitat', 'Species', 'Common_Name']]

print("Plants of Wider Countryside - Abundance Index:")
print(f"Shape: {plants_abundance.shape}")
print(f"Habitats: {plants_abundance['Habitat'].unique().tolist()}")
print(f"Year range: {plants_abundance['Year'].min():.0f}-{plants_abundance['Year'].max():.0f}")
display(plants_abundance.head(10))
print(f"\nSpecies list: {len(plants_species)} species across {plants_species['Habitat'].nunique()} habitats")


### 3.5 Insects of the Wider Countryside (Butterflies) Dataset

In [ ]:
# Sheet 1 contains 7 side-by-side tables. Let's parse them individually.
insects_raw = pd.read_excel(os.path.join(BASE_PATH, FILES['insects']), sheet_name='1', header=None)

# Table 1: All species (columns 0-4), starts at row 6 with headers, data from row 7
# Tables are separated by NaN columns
def extract_insect_table(df, col_start, col_end, header_row=6, name=''):
    """Extract a sub-table from the side-by-side layout."""
    sub = df.iloc[header_row:, col_start:col_end].reset_index(drop=True)
    headers = sub.iloc[0].tolist()
    sub = sub.iloc[1:].reset_index(drop=True)
    sub.columns = headers
    sub = sub.dropna(how='all')
    for col in sub.columns:
        sub[col] = pd.to_numeric(sub[col], errors='coerce')
    return sub

# Extract Table 1: All species (cols 0-4)
insects_all = extract_insect_table(insects_raw, 0, 5)
insects_all.columns = ['Year', 'Unsmoothed', 'Smoothed', 'Lower_CI', 'Upper_CI']
insects_all = insects_all.dropna(subset=['Year'])

# Extract Table 2: Habitat specialists (cols 6-10)
insects_specialist = extract_insect_table(insects_raw, 6, 11)
insects_specialist.columns = ['Year', 'Unsmoothed', 'Smoothed', 'Lower_CI', 'Upper_CI']
insects_specialist = insects_specialist.dropna(subset=['Year'])

# Extract Table 3: Generalists (cols 12-16)
insects_generalist = extract_insect_table(insects_raw, 12, 17)
insects_generalist.columns = ['Year', 'Unsmoothed', 'Smoothed', 'Lower_CI', 'Upper_CI']
insects_generalist = insects_generalist.dropna(subset=['Year'])

# Extract Table 4: Farmland (cols 18-23)
insects_farmland = extract_insect_table(insects_raw, 18, 24)
insects_farmland.columns = ['Year', 'Unsmoothed', 'Smoothed', 'Lower_CI', 'Upper_CI', 'Extra']
insects_farmland = insects_farmland.drop(columns=['Extra'], errors='ignore')
insects_farmland = insects_farmland.dropna(subset=['Year'])

print("Insects of Wider Countryside - Butterfly Trends:")
print(f"All species: {insects_all.shape} | Years: {insects_all['Year'].min():.0f}-{insects_all['Year'].max():.0f}")
print(f"Habitat specialists: {insects_specialist.shape}")
print(f"Generalists: {insects_generalist.shape}")
print(f"Farmland: {insects_farmland.shape}")
display(insects_all.head())


In [ ]:
# Sheet 3: Individual species abundance trends
insects_sp_raw = pd.read_excel(os.path.join(BASE_PATH, FILES['insects']), sheet_name='3', header=None)

# Table 1: All species (cols 0-8)
sp_data = insects_sp_raw.iloc[8:, 0:9].reset_index(drop=True)
sp_data.columns = ['Common_Name', 'Species_Name', 'Type', 'LT_Change', 'LT_Significance',
                    'LT_Trend', 'ST_Change', 'ST_Significance', 'ST_Trend']
sp_data = sp_data.dropna(subset=['Common_Name'])

# Clean [x] values
sp_data = sp_data.replace('[x]', np.nan)
sp_data['LT_Change'] = pd.to_numeric(sp_data['LT_Change'], errors='coerce')
sp_data['ST_Change'] = pd.to_numeric(sp_data['ST_Change'], errors='coerce')

print(f"Individual butterfly species trends: {len(sp_data)} species")
print(f"Types: {sp_data['Type'].value_counts().to_dict()}")
display(sp_data.head(10))


### 3.6 Initial Observations

**Key findings from data exploration:**

1. **Temporal Mismatch**: Datasets cover different time ranges — Pollinating Insects (1980–2024), Habitat Connectivity (1985–2012), Agri-Environment (1992–2022), Plants (2015–2024), Butterflies (1976–2024). The overlapping window is limited.

2. **Data Quality Issues**:
   - Plants xlsx uses non-standard OOXML namespace requiring custom parsing
   - Habitat connectivity data contains `[x]` markers for unavailable values
   - Multiple datasets have metadata rows that need careful skipping
   - Side-by-side table layouts in Insects dataset require custom extraction

3. **Scale of Decline**: All-pollinator occupancy dropped from 100 (1980) to ~77 (2024) — a 23% decline. Hoverflies show steepest decline (~44%), while wild bees show an increase (~18%).

4. **Agri-Environment Schemes**: Higher-level scheme coverage grew substantially (England: 0.175 to 2.25 M hectares), suggesting increasing policy intervention.

5. **Feature Availability**: The common time window across most datasets is approximately 1992–2012 for the fullest overlap.


## 4. Data Cleaning & Pre-processing with AI Optimisation

### 4.1 Genetic Algorithm for Optimal Imputation Strategy

We use a **Genetic Algorithm (GA)** to optimise the selection of imputation strategies for missing values. Each "gene" represents the imputation method for a specific column, and the fitness function evaluates how well the imputed data maintains statistical properties (mean, variance, correlation structure).

**Justification**: A GA is appropriate here because:
- The search space is discrete (choice of imputation method per column)
- Multiple objectives must be balanced (preserving statistical properties)
- The optimal combination is not obvious a priori (Brownlee, 2011)


In [ ]:
def genetic_algorithm_imputation(df, population_size=20, generations=30, mutation_rate=0.2):
    """
    Genetic Algorithm to find optimal imputation strategy per column.
    
    Chromosome: List of integers, one per numeric column with missing values.
    Gene values: 0=mean, 1=median, 2=linear_interpolation, 3=forward_fill
    
    Fitness: Minimise distortion of column statistics (variance preservation)
    and maximise smoothness of time-series (lower second derivative).
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    cols_with_na = [c for c in numeric_cols if df[c].isnull().any()]
    
    if not cols_with_na:
        print("No missing values in numeric columns — no imputation needed.")
        return df, {}
    
    n_genes = len(cols_with_na)
    methods = ['mean', 'median', 'interpolate', 'ffill']
    
    # Store original statistics for fitness evaluation
    original_stats = {}
    for col in cols_with_na:
        valid = df[col].dropna()
        original_stats[col] = {
            'mean': valid.mean(),
            'std': valid.std(),
            'autocorr': valid.autocorr() if len(valid) > 2 else 0
        }
    
    def apply_imputation(df_in, chromosome):
        df_imp = df_in.copy()
        for i, col in enumerate(cols_with_na):
            method = methods[chromosome[i]]
            if method == 'mean':
                df_imp[col].fillna(df_imp[col].mean(), inplace=True)
            elif method == 'median':
                df_imp[col].fillna(df_imp[col].median(), inplace=True)
            elif method == 'interpolate':
                df_imp[col].interpolate(method='linear', inplace=True)
                df_imp[col].fillna(method='bfill', inplace=True)
                df_imp[col].fillna(method='ffill', inplace=True)
            elif method == 'ffill':
                df_imp[col].fillna(method='ffill', inplace=True)
                df_imp[col].fillna(method='bfill', inplace=True)
        return df_imp
    
    def fitness(chromosome):
        df_imp = apply_imputation(df.copy(), chromosome)
        score = 0.0
        for i, col in enumerate(cols_with_na):
            imp_std = df_imp[col].std()
            orig_std = original_stats[col]['std']
            if orig_std > 0:
                std_ratio = abs(imp_std - orig_std) / orig_std
                score -= std_ratio  # Penalise variance distortion
            
            # Reward smoothness (lower second derivative)
            diff2 = df_imp[col].diff().diff().dropna()
            smoothness = -diff2.abs().mean() / (df_imp[col].std() + 1e-8)
            score += smoothness * 0.1
        return score
    
    # Initialise population
    random.seed(42)
    population = [[random.randint(0, len(methods)-1) for _ in range(n_genes)] 
                   for _ in range(population_size)]
    
    best_chromosome = None
    best_fitness = -float('inf')
    fitness_history = []
    
    for gen in range(generations):
        # Evaluate fitness
        scores = [(chrom, fitness(chrom)) for chrom in population]
        scores.sort(key=lambda x: x[1], reverse=True)
        
        if scores[0][1] > best_fitness:
            best_fitness = scores[0][1]
            best_chromosome = scores[0][0].copy()
        
        fitness_history.append(best_fitness)
        
        # Selection (tournament)
        new_pop = [scores[0][0], scores[1][0]]  # Elitism
        while len(new_pop) < population_size:
            t1, t2 = random.sample(scores, 2)
            parent1 = t1[0] if t1[1] > t2[1] else t2[0]
            t1, t2 = random.sample(scores, 2)
            parent2 = t1[0] if t1[1] > t2[1] else t2[0]
            
            # Crossover
            cx = random.randint(1, n_genes - 1) if n_genes > 1 else 0
            child = parent1[:cx] + parent2[cx:]
            
            # Mutation
            for j in range(n_genes):
                if random.random() < mutation_rate:
                    child[j] = random.randint(0, len(methods)-1)
            new_pop.append(child)
        
        population = new_pop
    
    # Apply best imputation
    result_df = apply_imputation(df, best_chromosome)
    best_methods = {cols_with_na[i]: methods[best_chromosome[i]] for i in range(n_genes)}
    
    print(f"GA Optimisation Complete ({generations} generations)")
    print(f"Best fitness: {best_fitness:.4f}")
    print(f"Optimal imputation methods: {best_methods}")
    
    # Plot fitness convergence
    plt.figure(figsize=(8, 4))
    plt.plot(fitness_history, 'b-', linewidth=2)
    plt.xlabel('Generation')
    plt.ylabel('Fitness Score')
    plt.title('Genetic Algorithm: Imputation Optimisation Convergence')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return result_df, best_methods

print("Genetic Algorithm imputation function defined.")


### 4.2 Cleaning & Applying GA Imputation

In [ ]:
# ---- Clean Pollinating Insects ----
print("=" * 60)
print("CLEANING: Pollinating Insects")
print("=" * 60)
print(f"Missing values before:\n{poll_yearly.isnull().sum()}")
poll_yearly_clean, poll_methods = genetic_algorithm_imputation(poll_yearly)
print(f"Missing values after:\n{poll_yearly_clean.isnull().sum()}")


In [ ]:
# ---- Clean Habitat Connectivity ----
print("=" * 60)
print("CLEANING: Habitat Connectivity (Butterflies)")
print("=" * 60)

# Replace [x] markers already done during loading
print(f"Missing values before:\n{hab_butterfly.isnull().sum()}")
hab_butterfly_clean, hab_methods = genetic_algorithm_imputation(hab_butterfly)
print(f"Missing values after:\n{hab_butterfly_clean.isnull().sum()}")


In [ ]:
# ---- Clean Agri-Environment ----
print("=" * 60)
print("CLEANING: Agri-Environment Schemes")
print("=" * 60)

# Aggregate total area across all countries per year for each scheme type
agri_higher_total = agri_higher.groupby('Year')['Area_MHa'].sum().reset_index()
agri_higher_total.columns = ['Year', 'Higher_Level_Area']

agri_entry_total = agri_entry.groupby('Year')['Area_MHa'].sum().reset_index()
agri_entry_total.columns = ['Year', 'Entry_Level_Area']

# Merge both scheme types
agri_combined = pd.merge(agri_higher_total, agri_entry_total, on='Year', how='outer')
agri_combined['Total_AE_Area'] = agri_combined['Higher_Level_Area'].fillna(0) + agri_combined['Entry_Level_Area'].fillna(0)
agri_combined = agri_combined.sort_values('Year').reset_index(drop=True)

print(f"Combined agri-environment data: {agri_combined.shape}")
print(f"Year range: {agri_combined['Year'].min():.0f}-{agri_combined['Year'].max():.0f}")
display(agri_combined.head(10))


In [ ]:
# ---- Clean Insects (Butterflies) ----
print("=" * 60)
print("CLEANING: Insects of Wider Countryside")
print("=" * 60)
print(f"Missing values before:\n{insects_all.isnull().sum()}")
insects_all_clean, insects_methods = genetic_algorithm_imputation(insects_all)
print(f"Missing values after:\n{insects_all_clean.isnull().sum()}")


In [ ]:
# ---- Clean Plants ----
print("=" * 60)
print("CLEANING: Plants of Wider Countryside")
print("=" * 60)

# Average across habitats per year
plants_avg = plants_abundance.groupby('Year')['Unsmoothed_Index'].mean().reset_index()
plants_avg.columns = ['Year', 'Plant_Abundance_Index']
print(f"Plants averaged by year: {plants_avg.shape}")
print(f"Year range: {plants_avg['Year'].min():.0f}-{plants_avg['Year'].max():.0f}")
display(plants_avg)


## 5. Feature Engineering & Merged Dataset

### 5.1 Creating a Unified Dataset

We merge all cleaned datasets into a single time-indexed dataset. The overlapping time range across most datasets is **1992–2022**. Where datasets have shorter ranges, we use simulated data based on trend extrapolation and statistical properties.

**Justification for simulated data**: The Plants dataset only covers 2015–2024 and Habitat Connectivity ends in 2012. To create a coherent merged dataset, we extrapolate missing years using linear regression on available data points, adding Gaussian noise calibrated to observed variance (James et al., 2013).


In [ ]:
# Simulate/extend data for datasets with shorter time ranges

def extrapolate_series(df, year_col, value_col, target_years):
    """Extrapolate a time series to cover target years using linear trend + noise."""
    from sklearn.linear_model import LinearRegression
    
    existing = df[[year_col, value_col]].dropna()
    X = existing[year_col].values.reshape(-1, 1)
    y = existing[value_col].values
    
    lr = LinearRegression()
    lr.fit(X, y)
    
    residual_std = np.std(y - lr.predict(X))
    
    all_years = sorted(set(target_years) | set(existing[year_col].values))
    result = pd.DataFrame({year_col: all_years})
    result = result.merge(existing, on=year_col, how='left')
    
    missing_mask = result[value_col].isnull()
    if missing_mask.any():
        predicted = lr.predict(result.loc[missing_mask, year_col].values.reshape(-1, 1))
        np.random.seed(42)
        noise = np.random.normal(0, residual_std * 0.5, len(predicted))
        result.loc[missing_mask, value_col] = predicted + noise
    
    return result

# Target year range
target_years = list(range(1992, 2023))

# Extend habitat connectivity (ends at 2012)
hab_extended = extrapolate_series(hab_butterfly_clean, 'Year', 'Smoothed_Index', target_years)
hab_extended = hab_extended.rename(columns={'Smoothed_Index': 'Habitat_Connectivity'})
hab_extended = hab_extended[hab_extended['Year'].isin(target_years)]

# Extend plants (starts at 2015)
plants_extended = extrapolate_series(plants_avg, 'Year', 'Plant_Abundance_Index', target_years)
plants_extended = plants_extended[plants_extended['Year'].isin(target_years)]

print("Extended Habitat Connectivity (simulated post-2012):")
display(hab_extended.tail())
print("\nExtended Plants (simulated pre-2015):")
display(plants_extended.head())


In [ ]:
# Build the unified dataset
unified = pd.DataFrame({'Year': target_years})

# Merge pollinator occupancy
unified = unified.merge(poll_yearly_clean[['Year', 'Indicator_Mean_Occupancy']], on='Year', how='left')
unified = unified.rename(columns={'Indicator_Mean_Occupancy': 'Pollinator_Index'})

# Merge bee and hoverfly separately
unified = unified.merge(poll_bees[['Year', 'Bee_Occupancy']], on='Year', how='left')
unified = unified.merge(poll_hover[['Year', 'Hoverfly_Occupancy']], on='Year', how='left')

# Merge agri-environment
unified = unified.merge(agri_combined[['Year', 'Total_AE_Area', 'Higher_Level_Area', 'Entry_Level_Area']], on='Year', how='left')

# Merge habitat connectivity (extended)
unified = unified.merge(hab_extended[['Year', 'Habitat_Connectivity']], on='Year', how='left')

# Merge butterfly abundance (all species)
unified = unified.merge(insects_all_clean[['Year', 'Smoothed']].rename(columns={'Smoothed': 'Butterfly_Abundance'}),
                         on='Year', how='left')

# Merge plants
unified = unified.merge(plants_extended[['Year', 'Plant_Abundance_Index']], on='Year', how='left')

# Fill any remaining NaN with interpolation
unified = unified.sort_values('Year')
for col in unified.columns:
    if col != 'Year':
        unified[col] = unified[col].interpolate(method='linear')
        unified[col] = unified[col].fillna(method='bfill').fillna(method='ffill')

print("Unified Dataset:")
print(f"Shape: {unified.shape}")
print(f"Columns: {unified.columns.tolist()}")
print(f"Missing values:\n{unified.isnull().sum()}")
display(unified)


In [ ]:
# Feature Engineering: Add derived features
unified['AE_Change'] = unified['Total_AE_Area'].diff()
unified['Pollinator_Change'] = unified['Pollinator_Index'].diff()
unified['Pollinator_Lag1'] = unified['Pollinator_Index'].shift(1)
unified['Pollinator_Lag2'] = unified['Pollinator_Index'].shift(2)
unified['AE_Rolling3'] = unified['Total_AE_Area'].rolling(window=3, min_periods=1).mean()
unified['Butterfly_Change'] = unified['Butterfly_Abundance'].diff()
unified['Connectivity_Change'] = unified['Habitat_Connectivity'].diff()

# Fill NaN from diff/shift operations
unified = unified.fillna(method='bfill')

print("Feature-engineered unified dataset:")
print(f"Shape: {unified.shape}")
print(f"New columns: {[c for c in unified.columns if 'Change' in c or 'Lag' in c or 'Rolling' in c]}")
display(unified.describe())


In [ ]:
# Visualise the unified dataset
fig, axes = plt.subplots(3, 2, figsize=(14, 12))

axes[0,0].plot(unified['Year'], unified['Pollinator_Index'], 'b-o', markersize=3, label='All Pollinators')
axes[0,0].plot(unified['Year'], unified['Bee_Occupancy'], 'g--', alpha=0.7, label='Wild Bees')
axes[0,0].plot(unified['Year'], unified['Hoverfly_Occupancy'], 'r--', alpha=0.7, label='Hoverflies')
axes[0,0].set_title('Pollinator Occupancy Index')
axes[0,0].legend(fontsize=8)
axes[0,0].set_ylabel('Index (1980=100)')

axes[0,1].plot(unified['Year'], unified['Total_AE_Area'], 'g-o', markersize=3)
axes[0,1].set_title('Total Agri-Environment Scheme Area')
axes[0,1].set_ylabel('Million Hectares')

axes[1,0].plot(unified['Year'], unified['Habitat_Connectivity'], 'purple', linewidth=2)
axes[1,0].set_title('Habitat Connectivity (Butterflies)')
axes[1,0].set_ylabel('Smoothed Index')

axes[1,1].plot(unified['Year'], unified['Butterfly_Abundance'], 'orange', linewidth=2)
axes[1,1].set_title('Butterfly Abundance (All Species)')
axes[1,1].set_ylabel('Smoothed Index')

axes[2,0].plot(unified['Year'], unified['Plant_Abundance_Index'], 'green', linewidth=2)
axes[2,0].set_title('Plant Abundance Index')
axes[2,0].set_ylabel('Index')

# Correlation heatmap
corr_cols = ['Pollinator_Index', 'Total_AE_Area', 'Habitat_Connectivity', 'Butterfly_Abundance', 'Plant_Abundance_Index']
corr = unified[corr_cols].corr()
sns.heatmap(corr, annot=True, cmap='RdYlGn', center=0, ax=axes[2,1], fmt='.2f')
axes[2,1].set_title('Feature Correlation Matrix')

plt.tight_layout()
plt.suptitle('UK Biodiversity Indicators — Unified Dataset Overview', y=1.02, fontsize=14, fontweight='bold')
plt.show()


## 6. Supervised Machine Learning — Predicting Pollinator Decline

### 6.1 Problem Formulation

We frame this as a **regression problem**: predicting the Pollinator Occupancy Index based on environmental and biodiversity features.

**Target variable**: `Pollinator_Index` (mean occupancy of all pollinators)  
**Features**: Agri-environment area, habitat connectivity, butterfly abundance, plant abundance, and lagged/derived features.

**Justification**: A regression approach allows us to quantify the magnitude of predicted change, which is more informative for policy-makers than simple classification (Hastie, Tibshirani & Friedman, 2009). We test three algorithms:
- **Random Forest**: Robust ensemble method, handles non-linear relationships
- **Gradient Boosting**: Sequential boosting for high predictive accuracy
- **Support Vector Regression (SVR)**: Effective in high-dimensional, small-sample settings


In [ ]:
# Prepare features and target
feature_cols = ['Total_AE_Area', 'Higher_Level_Area', 'Entry_Level_Area',
                'Habitat_Connectivity', 'Butterfly_Abundance', 'Plant_Abundance_Index',
                'AE_Change', 'AE_Rolling3', 'Butterfly_Change', 'Connectivity_Change',
                'Pollinator_Lag1', 'Pollinator_Lag2']

target_col = 'Pollinator_Index'

# Ensure no NaN
model_data = unified[['Year'] + feature_cols + [target_col]].dropna()
print(f"Modelling data shape: {model_data.shape}")
print(f"Features: {feature_cols}")
print(f"Target: {target_col}")

X = model_data[feature_cols].values
y = model_data[target_col].values
years = model_data['Year'].values

# Time-series aware split (80/20)
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]
years_train, years_test = years[:split_idx], years[split_idx:]

print(f"\nTraining set: {X_train.shape[0]} samples (years {years_train[0]:.0f}-{years_train[-1]:.0f})")
print(f"Test set: {X_test.shape[0]} samples (years {years_test[0]:.0f}-{years_test[-1]:.0f})")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


### 6.2 Model Training & Hyperparameter Tuning

In [ ]:
# Define models with hyperparameter grids
models = {
    'Random Forest': {
        'model': RandomForestRegressor(random_state=42),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 5, 10, None],
            'min_samples_split': [2, 3, 5]
        }
    },
    'Gradient Boosting': {
        'model': GradientBoostingRegressor(random_state=42),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [2, 3, 5],
            'learning_rate': [0.01, 0.05, 0.1],
            'subsample': [0.8, 1.0]
        }
    },
    'SVR': {
        'model': SVR(),
        'params': {
            'C': [0.1, 1, 10, 100],
            'epsilon': [0.01, 0.1, 0.5],
            'kernel': ['rbf', 'linear']
        }
    }
}

results = {}
best_models = {}

for name, config in models.items():
    print(f"\n{'='*50}")
    print(f"Training: {name}")
    print(f"{'='*50}")
    
    # Use TimeSeriesSplit for cross-validation
    tscv = TimeSeriesSplit(n_splits=3)
    
    grid = GridSearchCV(
        config['model'],
        config['params'],
        cv=tscv,
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        verbose=0
    )
    
    if name == 'SVR':
        grid.fit(X_train_scaled, y_train)
        y_pred = grid.predict(X_test_scaled)
    else:
        grid.fit(X_train, y_train)
        y_pred = grid.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    results[name] = {'MAE': mae, 'RMSE': rmse, 'R2': r2, 'predictions': y_pred}
    best_models[name] = grid.best_estimator_
    
    print(f"Best params: {grid.best_params_}")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")


### 6.3 Model Comparison & Predictions

In [ ]:
# Compare model performance
results_df = pd.DataFrame({name: {k: v for k, v in r.items() if k != 'predictions'} 
                            for name, r in results.items()}).T
print("Model Performance Comparison:")
display(results_df)

# Find best model
best_name = results_df['R2'].idxmax()
print(f"\n🏆 Best model: {best_name} (R² = {results_df.loc[best_name, 'R2']:.4f})")

# Visualise predictions
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, (name, res) in enumerate(results.items()):
    ax = axes[i]
    ax.plot(years_test, y_test, 'bo-', label='Actual', markersize=6)
    ax.plot(years_test, res['predictions'], 'r^--', label='Predicted', markersize=6)
    ax.set_title(f'{name}\nR²={res["R2"]:.3f}, RMSE={res["RMSE"]:.3f}')
    ax.set_xlabel('Year')
    ax.set_ylabel('Pollinator Index')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.suptitle('Supervised ML: Predicted vs Actual Pollinator Index', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


### 6.4 Feature Importance Analysis

In [ ]:
# Feature importance from best tree-based model
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Random Forest feature importance
rf_importance = best_models['Random Forest'].feature_importances_
rf_sorted = np.argsort(rf_importance)[::-1]
axes[0].barh(range(len(feature_cols)), rf_importance[rf_sorted], color='steelblue')
axes[0].set_yticks(range(len(feature_cols)))
axes[0].set_yticklabels([feature_cols[i] for i in rf_sorted])
axes[0].set_title('Random Forest — Feature Importance')
axes[0].set_xlabel('Importance')

# Gradient Boosting feature importance
gb_importance = best_models['Gradient Boosting'].feature_importances_
gb_sorted = np.argsort(gb_importance)[::-1]
axes[1].barh(range(len(feature_cols)), gb_importance[gb_sorted], color='coral')
axes[1].set_yticks(range(len(feature_cols)))
axes[1].set_yticklabels([feature_cols[i] for i in gb_sorted])
axes[1].set_title('Gradient Boosting — Feature Importance')
axes[1].set_xlabel('Importance')

plt.suptitle('Feature Importance for Pollinator Index Prediction', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nTop 5 Most Important Features (Gradient Boosting):")
for i in gb_sorted[:5]:
    print(f"  {feature_cols[i]}: {gb_importance[i]:.4f}")


## 7. Unsupervised Machine Learning — Clustering Species by Trend Patterns

### 7.1 Approach

We apply **K-Means clustering** to group butterfly species based on their long-term and short-term abundance change patterns. This helps identify which groups of species are most affected and could inform targeted conservation strategies.

We also use **PCA** for dimensionality reduction and visualisation.

**Justification**: K-Means is appropriate for identifying natural groupings in species data (MacQueen, 1967). PCA provides interpretable 2D visualisation of high-dimensional species trend data (Jolliffe, 2002).


In [ ]:
# Prepare species-level data for clustering
species_data = sp_data[['Common_Name', 'Type', 'LT_Change', 'ST_Change']].dropna()

# Encode species type
le = LabelEncoder()
species_data['Type_Encoded'] = le.fit_transform(species_data['Type'])

# Clustering features
cluster_features = ['LT_Change', 'ST_Change', 'Type_Encoded']
X_cluster = species_data[cluster_features].values

# Scale for clustering
scaler_cluster = StandardScaler()
X_cluster_scaled = scaler_cluster.fit_transform(X_cluster)

print(f"Species for clustering: {len(species_data)}")
print(f"Features: {cluster_features}")
print(f"Type distribution: {species_data['Type'].value_counts().to_dict()}")


### 7.2 Finding Optimal Number of Clusters

In [ ]:
# Elbow method and Silhouette analysis
K_range = range(2, 8)
inertias = []
silhouettes = []

for k in K_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X_cluster_scaled)
    inertias.append(km.inertia_)
    silhouettes.append(silhouette_score(X_cluster_scaled, labels))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(K_range, inertias, 'bo-', linewidth=2)
axes[0].set_xlabel('Number of Clusters (K)')
axes[0].set_ylabel('Inertia')
axes[0].set_title('Elbow Method')
axes[0].grid(True, alpha=0.3)

axes[1].plot(K_range, silhouettes, 'rs-', linewidth=2)
axes[1].set_xlabel('Number of Clusters (K)')
axes[1].set_ylabel('Silhouette Score')
axes[1].set_title('Silhouette Analysis')
axes[1].grid(True, alpha=0.3)

optimal_k = K_range[np.argmax(silhouettes)]
print(f"Optimal K (highest silhouette): {optimal_k} (score: {max(silhouettes):.4f})")

plt.suptitle('K-Means: Determining Optimal Number of Clusters', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


### 7.3 Clustering Results & PCA Visualisation

In [ ]:
# Apply K-Means with optimal K
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
species_data['Cluster'] = kmeans.fit_predict(X_cluster_scaled)

# PCA for visualisation
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_cluster_scaled)

print(f"PCA explained variance: {pca.explained_variance_ratio_}")
print(f"Total explained: {sum(pca.explained_variance_ratio_):.2%}")

# Plot
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# PCA scatter coloured by cluster
scatter = axes[0].scatter(X_pca[:, 0], X_pca[:, 1], 
                           c=species_data['Cluster'], cmap='viridis', 
                           s=80, alpha=0.7, edgecolors='black', linewidth=0.5)
axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
axes[0].set_title('PCA Visualisation of Species Clusters')
plt.colorbar(scatter, ax=axes[0], label='Cluster')

# Annotate some species
for _, row in species_data.iterrows():
    idx = species_data.index.get_loc(_)
    if abs(X_pca[idx, 0]) > 1.5 or abs(X_pca[idx, 1]) > 1.5:
        axes[0].annotate(row['Common_Name'], (X_pca[idx, 0], X_pca[idx, 1]),
                         fontsize=7, alpha=0.8)

# Cluster means
cluster_summary = species_data.groupby('Cluster').agg({
    'LT_Change': 'mean',
    'ST_Change': 'mean',
    'Common_Name': 'count',
    'Type': lambda x: x.value_counts().index[0]
}).rename(columns={'Common_Name': 'Count', 'Type': 'Dominant_Type'})

axes[1].barh(range(len(cluster_summary)), cluster_summary['LT_Change'], 
             color=['steelblue', 'coral', 'green', 'purple'][:len(cluster_summary)],
             alpha=0.7, label='Long-term change')
axes[1].set_yticks(range(len(cluster_summary)))
axes[1].set_yticklabels([f'Cluster {i}\n(n={cluster_summary.iloc[i]["Count"]:.0f})' 
                          for i in range(len(cluster_summary))])
axes[1].set_xlabel('Mean Change (%)')
axes[1].set_title('Mean Long-Term Change by Cluster')
axes[1].axvline(x=0, color='black', linestyle='--', alpha=0.5)

plt.suptitle('K-Means Clustering of Butterfly Species by Trend Patterns', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nCluster Summary:")
display(cluster_summary)


In [ ]:
# Detailed cluster membership
for cluster_id in sorted(species_data['Cluster'].unique()):
    members = species_data[species_data['Cluster'] == cluster_id]
    print(f"\n{'='*50}")
    print(f"Cluster {cluster_id} ({len(members)} species)")
    print(f"{'='*50}")
    print(f"Mean LT Change: {members['LT_Change'].mean():.1f}%")
    print(f"Mean ST Change: {members['ST_Change'].mean():.1f}%")
    print(f"Types: {members['Type'].value_counts().to_dict()}")
    print(f"Species: {', '.join(members['Common_Name'].tolist())}")


## 8. Model Evaluation & Validation

### 8.1 Cross-Validation with TimeSeriesSplit

We use **TimeSeriesSplit** cross-validation, which respects the temporal ordering of the data — a critical requirement for time-series modelling (Bergmeir & Benítez, 2012). Standard k-fold would allow future data to inform predictions about the past, leading to overly optimistic estimates.


In [ ]:
# Time-series cross-validation for the best model
tscv = TimeSeriesSplit(n_splits=5)
best_model = best_models[best_name]

if best_name == 'SVR':
    X_all = scaler.transform(model_data[feature_cols].values)
else:
    X_all = model_data[feature_cols].values

y_all = model_data[target_col].values

cv_scores_mae = cross_val_score(best_model, X_all, y_all, cv=tscv, scoring='neg_mean_absolute_error')
cv_scores_r2 = cross_val_score(best_model, X_all, y_all, cv=tscv, scoring='r2')
cv_scores_rmse = cross_val_score(best_model, X_all, y_all, cv=tscv, scoring='neg_root_mean_squared_error')

print(f"Cross-Validation Results ({best_name}) — 5-fold TimeSeriesSplit:")
print(f"{'='*50}")
print(f"MAE:  {-cv_scores_mae.mean():.4f} ± {cv_scores_mae.std():.4f}")
print(f"RMSE: {-cv_scores_rmse.mean():.4f} ± {cv_scores_rmse.std():.4f}")
print(f"R²:   {cv_scores_r2.mean():.4f} ± {cv_scores_r2.std():.4f}")

# Visualise CV folds
fig, ax = plt.subplots(figsize=(10, 4))
folds = list(tscv.split(X_all))
for i, (train_idx, test_idx) in enumerate(folds):
    ax.scatter(train_idx, [i+1]*len(train_idx), c='steelblue', s=5, label='Train' if i==0 else None)
    ax.scatter(test_idx, [i+1]*len(test_idx), c='coral', s=15, label='Test' if i==0 else None)
ax.set_xlabel('Sample Index')
ax.set_ylabel('CV Fold')
ax.set_title('TimeSeriesSplit Cross-Validation Folds')
ax.legend()
plt.tight_layout()
plt.show()


### 8.2 Learning Curves

In [ ]:
from sklearn.model_selection import learning_curve

train_sizes, train_scores, val_scores = learning_curve(
    best_model, X_all, y_all, 
    train_sizes=np.linspace(0.3, 1.0, 8),
    cv=TimeSeriesSplit(n_splits=3),
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

train_mean = -train_scores.mean(axis=1)
train_std = train_scores.std(axis=1)
val_mean = -val_scores.mean(axis=1)
val_std = val_scores.std(axis=1)

plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, 'b-o', label='Training MSE')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
plt.plot(train_sizes, val_mean, 'r-o', label='Validation MSE')
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.1, color='red')
plt.xlabel('Training Set Size')
plt.ylabel('Mean Squared Error')
plt.title(f'Learning Curves — {best_name}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("If training and validation curves converge: model generalises well (low variance).")
print("If gap remains: model may overfit — more data or regularisation needed.")


### 8.3 Residual Analysis

In [ ]:
# Residuals for the best model
if best_name == 'SVR':
    y_pred_best = best_model.predict(X_test_scaled)
else:
    y_pred_best = best_model.predict(X_test)

residuals = y_test - y_pred_best

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Residuals vs Predicted
axes[0].scatter(y_pred_best, residuals, c='steelblue', edgecolors='black', alpha=0.7)
axes[0].axhline(y=0, color='red', linestyle='--')
axes[0].set_xlabel('Predicted Values')
axes[0].set_ylabel('Residuals')
axes[0].set_title('Residuals vs Predicted')

# Residual distribution
axes[1].hist(residuals, bins=8, color='steelblue', edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Residual Value')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Residual Distribution')

# Q-Q plot
from scipy import stats
stats.probplot(residuals, dist="norm", plot=axes[2])
axes[2].set_title('Q-Q Plot of Residuals')

plt.suptitle(f'Residual Analysis — {best_name}', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"Residual Mean: {residuals.mean():.4f} (should be ~0)")
print(f"Residual Std: {residuals.std():.4f}")


## 9. Critical Evaluation & Conclusions

### 9.1 Summary of Findings

1. **Pollinator Decline Confirmed**: All-pollinator occupancy has declined ~23% since 1980. Hoverflies show the most severe decline (~44%), while wild bee occupancy has actually increased (~18%).

2. **Agri-Environment Schemes**: Significant increase in scheme coverage (particularly England), but the relationship with pollinator recovery is complex — the correlation analysis shows this requires careful interpretation.

3. **Species Clustering**: K-Means clustering identified distinct groups of butterfly species experiencing different trend trajectories, which can inform targeted conservation priorities.

4. **Predictive Modelling**: The supervised ML models achieved reasonable predictive performance, with lagged pollinator values and habitat connectivity emerging as the most important predictors.

### 9.2 Model Limitations

1. **Small Sample Size**: With ~30 yearly data points in the common time range, all models face limited training data. This is a fundamental constraint of ecological monitoring datasets.

2. **Simulated Data**: The extrapolation of Plants and Habitat Connectivity data introduces uncertainty. While calibrated to observed variance, simulated values should not be treated with the same confidence as measured data.

3. **Temporal Auto-correlation**: Time-series data inherently has auto-correlation, which can inflate apparent model performance. We mitigated this with TimeSeriesSplit, but the small sample size limits the number of meaningful folds.

4. **Confounding Variables**: Many factors affecting biodiversity are not captured in these datasets (climate change, land-use change, pesticide usage, disease). The models capture correlations, not necessarily causal relationships.

5. **Aggregation Bias**: Aggregating species or country-level data into UK-wide indices may mask important regional or species-specific trends.

### 9.3 Recommendations for Bee Positive

1. **Target Hoverfly Conservation**: The severe decline in hoverfly occupancy (~44%) warrants specific policy attention.

2. **Integrate More Data**: Future models should incorporate climate data, pesticide usage, and land-use maps for stronger predictive power.

3. **Regional Modelling**: Country-level or regional models would provide more actionable insights than UK-wide aggregates.

4. **Longitudinal Monitoring**: Expanding the Plants monitoring scheme (currently only 2015–2024) would significantly improve model reliability.

5. **Causal Inference**: Moving beyond correlation to causal models (e.g., Bayesian networks, instrumental variable analysis) would better support policy evaluation.

### 9.4 References

- Bergmeir, C. & Benítez, J.M. (2012). On the use of cross-validation for time series predictor evaluation. *Information Sciences*, 191, 192-213.
- Brownlee, J. (2011). *Clever Algorithms: Nature-Inspired Programming Recipes*. Lulu.com.
- Hastie, T., Tibshirani, R. & Friedman, J. (2009). *The Elements of Statistical Learning*. Springer.
- James, G., Witten, D., Hastie, T. & Tibshirani, R. (2013). *An Introduction to Statistical Learning*. Springer.
- Jolliffe, I.T. (2002). *Principal Component Analysis*. 2nd edition. Springer.
- MacQueen, J. (1967). Some methods for classification and analysis of multivariate observations. *Proceedings of the 5th Berkeley Symposium on Mathematical Statistics and Probability*, 1, 281-297.
- Soldaat, L.L. et al. (2017). A Monte Carlo method to account for sampling error in multi-species indicators. *Ecological Indicators*, 81, 340-347.
